# Profiling
---

Objectives:
- Learn how to find slow parts of your Python code.
- Learn how to assess memory consumption of your code.

Profiling is the method of assessing the computational resources of your code.
Since we are coding with Python, we will discuss the profiling of Python code.
We will, however, touch upon other languages.

## Setup

All the profilers discussed next are installed by running

In [1]:
!pip install -r requirements_dev.txt

## Function-Level Profiling: cProfile

Let's assume a simple task.
We want to sum integers in a list and return the sum.

The function is readily coded and available in `src/main_sum.py`.
Assuming that this is a time-critical function, we have coded two such functions with the same input-output behavior.
They return the same output for the same input and are thus called functionally equivalent.

A priori, we do not know which of the function is faster - and by how much.

To shed light on this, let's run the basic profiler that is shipped with Python: [cProfile](https://docs.python.org/3/library/profile.html).
It comes as a pre-installed module and is run via:

In [2]:
!python -m cProfile src/main_sum.py

res_1=50000005000000
res_2=50000005000000
         1119184 function calls (1055822 primitive calls) in 1.476 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       35    0.001    0.000    4.944    0.141 __init__.py:1(<module>)
    660/1    0.019    0.000    1.478    1.478 {built-in method builtins.exec}
        1    0.058    0.058    1.478    1.478 main_sum.py:1(<module>)
        5    0.000    0.000    1.043    0.209 definitions.py:1(<module>)
    318/1    0.001    0.000    0.680    0.680 <frozen importlib._bootstrap>:1349(_find_and_load)
    318/1    0.001    0.000    0.680    0.680 <frozen importlib._bootstrap>:1304(_find_and_load_unlocked)
    294/1    0.000    0.000    0.679    0.679 <frozen importlib._bootstrap>:911(_load_unlocked)
    262/1    0.001    0.000    0.679    0.679 <frozen importlib._bootstrap_external>:993(exec_module)
    869/3    0.002    0.000    0.679    0.226 <frozen importlib._bootstrap>:480(_call_

### Interpreting Results

After terminating, and subsequent to the code's output, it outputs statistics about the code's functions.
Since this is a function-level profiler, the statistics are per function:
- `ncalls`: number of calls of this function
- `totime`: total time [s] in function without sub-functions
- `cumtime`: time [s] in function with sub-functions

Further, there are `percall` columns after statistics for the stistics normalized to the number of calls `ncalls`.
That may be of interest since an already fast function may still be the bottleneck if called many times.
In this case, `percall` would be small indicating a difficulty to shape more time of this function, and signaling us that it may be good to look for ways to reduce the number of function calls. 

We can also sort the results by one of the columns.
For example for `tottime` via:

```bash
python -m cProfile -s tottime src/main_sum.py 
```

Back to our initial question, we see that `sum_numbers_for` requires a time of 0.178 s (`cumtime`) and `sum_numbers_list` only 0.058 s.
The latter is clearly faster - in fact by a factor:

In [3]:
0.178/0.058

3.068965517241379

### Visualizing Results

Sometimes your question may be less precise: instead of wanting to know which of two functions is faster, you may just wander why your code is slow.
While for simple code, our previous method works as well, there are GUIs that help you get a feeling for time-bottlenecks in your code.

In general, profiling and exploring with a GUI is separated into two processes, by writing the profiling results to a file first.
We will thus write profiling results into, say, `tmp/main_sum.profile`.

In [4]:
!python -m cProfile -o tmp/main_sum.profile src/main_sum.py

res_1=50000005000000
res_2=50000005000000


There are several GUIs available to show profiling data.
One of them is [snakeviz](https://jiffyclub.github.io/snakeviz/).
You can graphically view the profile by running in your terminal:

```bash
snakeviz tmp/main_sum.profile
```

This will start a webserver and you have to stop this cell manually from running.

In [5]:
# !snakeviz tmp/main_sum.profile

The output looks like the image below and is browsable.

![snakeviz output](img/snakeviz.png)

## Line-by-line Profiler: Scalene

Sometimes, functions are too coarse to get a feeling for where time is lost.
This is addressed by so-called line-by-line profilers that provide timing statistics per code line.

Once of them is [Scalene](https://github.com/plasma-umass/scalene).
Features of Scalene are that it also profiles GPU and memory usage.
It also comes with a [VS Code plugin](https://marketplace.visualstudio.com/items?itemName=EmeryBerger.scalene).

Basic profiling of the compete code is run via:

In [6]:
!scalene run src/main_sum.py

res_1=50000005000000
res_2=50000005000000


The GUI is started by running in the terminal

```bash
scalene view
```
which again starts a webserver.

The result is shown below.

![Scalene result](img/scalene.png)

CPU time, or simply `TIME` in the picture above, is given in terms of:
- Python: CPU time running Python bytecode by the Python interpreter
- native: time in native code, such as C, C++, Rust, or other extensions. Much of SciPy or NumPy is written in such extensions.
- system: time in syscalls of the OS. Examples are read/write of files, memory allocations, waiting for locks.

An interesting feature is that we can also see memory intensive operations like the allocation of memory when creating the list of `int` values in line 22.

Further, the line profiler also returns a function-level profile.

While an advantage of this line-by-line profiler is that it does not need annotation of code, it provides annotation of functions via `@profile` to profile certain functions, only.
This is great if you want to dig deeper in a certain part of the code, only.

> **_Note:_** By contrast to the next presented profiler, you do not need to import `profile`.

## Line-by-line Profiler: line_profiler

Another classic is the [line_profiler](https://kernprof.readthedocs.io/en/latest/).
In its basic usage it requires annotation of the functions that need to be profiled.
Further, we need to include `profile` from the `line_profiler`.
For a function `f()`, the code would look like:

```python
from line_profiler import profile

@profile
def f():
    ...
```

When calling the profiler, make sure to set `LINE_PROFILE=1` in your shell as this activates the profiler.

```bash
LINE_PROFILE=1 python src/main_sum.py
```

The results are shown via running in the terminal:

```bash
python -m line_profiler -rtmz profile_output.lprof
```

Output is shown in the terminal.
For the case where we decorated the functions `sum_numbers_for` and `sum_numbers_list` with `@profile`, we obtain:

```bash
Timer unit: 1e-06 s

Total time: 0.056206 s
File: /Users/mfuegger/Github/profiling/src/main_sum.py
Function: sum_numbers_list at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    13                                           @profile                                     
    14                                           def sum_numbers_list(nums: list[int]) -> int:
    15         1          0.0      0.0      0.0      total: int = 0                           
    16         1      56204.0  56204.0    100.0      total = sum(list(nums))                  
    17         1          2.0      2.0      0.0      return total                             


Total time: 4.79964 s
File: /Users/mfuegger/Github/profiling/src/main_sum.py
Function: sum_numbers_for at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     5                                           @profile                                    
     6                                           def sum_numbers_for(nums: list[int]) -> int:
     7                                               """Sum integers within list."""         
     8         1          1.0      1.0      0.0      total: int = 0                          
     9  10000001    2334964.0      0.2     48.6      for num in nums:                        
    10  10000000    2464669.0      0.2     51.4          total += num                        
    11         1          3.0      3.0      0.0      return total                            


  0.06 seconds - /Users/mfuegger/Github/profiling/src/main_sum.py:13 - sum_numbers_list
  4.80 seconds - /Users/mfuegger/Github/profiling/src/main_sum.py:5 - sum_numbers_for
```

### Autoprofiling

Since decorating functions can be tedious and we often want to profile code without changing it,
we can also simply run: 

```bash
python -m kernprof -lv -p src/main_sum.py src/main_sum.py
```

This results in:

```bash
res_1=50000005000000
res_2=50000005000000
Wrote profile results to 'main_sum.py.lprof'
Timer unit: 1e-06 s

Total time: 4.38062 s
File: src/main_sum.py
Function: sum_numbers_for at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     6                                           def sum_numbers_for(nums: list[int]) -> int:
     7                                               """Sum integers within list."""
     8         1          0.0      0.0      0.0      total: int = 0
     9  10000001    2197500.0      0.2     50.2      for num in nums:
    10  10000000    2183113.0      0.2     49.8          total += num
    11         1          3.0      3.0      0.0      return total

Total time: 0.056563 s
File: src/main_sum.py
Function: sum_numbers_list at line 14

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    14                                           def sum_numbers_list(nums: list[int]) -> int:
    15         1          1.0      1.0      0.0      total: int = 0
    16         1      56560.0  56560.0    100.0      total = sum(list(nums))
    17         1          2.0      2.0      0.0      return total

Total time: 6.52508 s
File: src/main_sum.py
Function: main at line 20

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    20                                           def main() -> None:
    21         1          0.0      0.0      0.0      N: int = 10_000_000
    22         1      77253.0  77253.0      1.2      numbers: list[int] = list(range(1, N + 1))
    23                                           
    24         1    6391206.0 6.39e+06     97.9      res_1 = sum_numbers_for(numbers)
    25         1      56584.0  56584.0      0.9      res_2 = sum_numbers_list(numbers)
    26                                           
    27         1         37.0     37.0      0.0      print(f"{res_1=}\n{res_2=}")
```

# memory_profiler

Memory profiles over time can be generated via:

```bash
mprof run src/main_sum.py
```

and plotted via

```bash
mprof plot
```

Plotting can called during an mprof run, so that we see the current state.

In our case, the memory profile looks like this:

![mprof memory profile](img/mprof.png)


### The Garbage Collector

You can also run the memory profiler on a more interesting program from a memory perspective.
The code is available at `src/large_memory.py`.

To see the code in action, run a memory profile and observe the triggering of the Python garbage collector (gc).

![mprof of src/large_memory.py](img/mprof_large_mem.png)

Since significant numbers of objects have to be freed, the gc cycles halt the Python program for a significant mount of time - a time bottleneck that is sometimes missed from otherwise efficient code.
Indeed gc runs can be seconds and minutes for large object numbers.

# Multithreading and processing: Austin

Of particular mention is [Austin](https://github.com/P403n1x87/austin) that can handle async, threads, and multiprocessing.
It does this by sampling the CPython interpreter state, determining what it is currently doing.

Austin can do lots of things and we only show its basic profiling capabilities, here.
You can run it in the terminal via:

```bash
sudo austin -o tmp/main_sum.austin python src/main_sum.py
```

in MacOS runnign as superuser is required.
This results in:

```bash
              _   _      
 __ _ _  _ __| |_(_)_ _  
/ _` | || (_-<  _| | ' \ 
\__,_|\_,_/__/\__|_|_||_| 4.0.0 [clang 17.0.0]

🐍 Python version: 3.12.11

📈 Sampling Statistics

      Total duration . . . . . . 0.35s
      Average sampling rate  . . 7.84 kHz
      Error rate . . . . . . . . 0/2730 (0.00%)
```

You can also increase the sampling rate if you need finer granularity.

We are inspecting the results via an [Austin VSCode extension](https://github.com/P403n1x87/austin-vscode):
- Click in the Terminal view on `Flame Graph`
- Open the austin file.
- On the left sidebar select the Austin logo.
- By navigating to the main module, you can see a line-by-line statistics such as:

![Austin profile](img/austin-vscode.png)